In [6]:
import pandas as pd
import numpy as np
from scipy import stats


In [4]:
df = pd.read_csv('../data/data_all_surfaces.csv')
df.columns

Index(['HI', 'SK', 'KUR', 'DSK', 'DKUR', 'Surface'], dtype='object')

In [19]:
def perform_welch_ttest(df, group1_name, group2_name, metrics=['HI', 'SK', 'KUR', 'DSK', 'DKUR']):
    """
    Perform Welch's t-test between two groups for multiple metrics.
    
    Parameters:
    df: DataFrame containing our data
    group1_name: Name of first group in Surface column
    group2_name: Name of second group in Surface column
    metrics: List of metrics to compare
    
    Returns:
    Dictionary of results with significance symbols
    """
    results = {}
    
    for metric in metrics:
        group1 = df[df['Surface'] == group1_name][metric]
        group2 = df[df['Surface'] == group2_name][metric]
        
        if len(group1) > 0 and len(group2) > 0:
            t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
            
            # Convert p-value to significance symbol
            if p_val < 0.01:
                results[metric] = "**"
            elif p_val < 0.05:
                results[metric] = "*"
            else:
                results[metric] = "ns"
        else:
            results[metric] = "NA"
    
    return results

def create_comparison_table(df):
    """
    Create a table of Welch's t-test results for all required comparisons.
    """
    # Define the comparisons using your actual group names
    comparisons = [
        ('Earth (hyper arid)', 'Earth (humid)', "Hyper-arid Earth – Humid Earth"),
        ('Mars', 'Earth (humid)', "All Mars – Humid Earth"),
        ('Mars', 'Earth (hyper arid)', "All Mars – Hyper-arid Earth"),
        ('Mars', 'Fractal Surface', "All Mars – Fractal"),
        ('Moon', 'Fractal Surface', "Moon – Fractal"),
        ('Moon', 'Earth (hyper arid)', "Moon – Hyper-arid Earth"),
        ('Mars', 'Moon', "All Mars – Moon"),
        ('Earth_glaciated', 'Moon', "Earth Glaciated – Moon"),
        ('Earth_non_glaciated', 'Moon', "Earth Nonglaciated – Moon"),
        ('Mars_high_crater', 'Moon', "High crater↑(B) – Moon"),
        ('Mars_low_vn', 'Moon', "Low VN↓(C) – Moon"),
        ('Mars_low_crater', 'Moon', "Low crater↓(A) – Moon"),
        ('Mars_low_crater', 'Mars_high_crater', "Low crater↓(A) –High crater↑(B)"),
        ('Mars_high_vn', 'Mars_low_vn', "High VN↑(D) – Low VN↓(C)"),
        ('Mars_high_vn', 'Moon', "High VN↑(D) – Moon"),
        ('model_fluvial', 'Moon', "Model fluvial – Moon")
    ]
    
    # Create results table
    results = []
    for group1, group2, description in comparisons:
        row = perform_welch_ttest(df, group1, group2)
        row['Row'] = len(results) + 1  # Add row number
        row['Comparing Groups'] = description
        results.append(row)
    
    # Convert to DataFrame and format
    results_df = pd.DataFrame(results)
    
    # Reorder columns to put Row first
    column_order = ['Row', 'Comparing Groups', 'HI', 'SK', 'KUR', 'DSK', 'DKUR']
    results_df = results_df[column_order]
    
    return results_df


In [20]:
# recreate Table 2 in the paper
results_table = create_comparison_table(df)
print(results_table.to_string(index=False))

 Row                Comparing Groups HI SK KUR DSK DKUR
   1  Hyper-arid Earth – Humid Earth ** **  **  **   **
   2          All Mars – Humid Earth ** **  **  **   **
   3     All Mars – Hyper-arid Earth ** **  **  **   **
   4              All Mars – Fractal ** **  **  **   **
   5                  Moon – Fractal ** **  **  **   **
   6         Moon – Hyper-arid Earth ** **  **  **   **
   7                 All Mars – Moon ns  *   *  ns   **
   8          Earth Glaciated – Moon  * ns  ns   *    *
   9       Earth Nonglaciated – Moon ns **   *  ns   ns
  10          High crater↑(B) – Moon ns ns  ns  ns   ns
  11               Low VN↓(C) – Moon ns ns  ns  ns   ns
  12           Low crater↓(A) – Moon ** **  **  **   ns
  13 Low crater↓(A) –High crater↑(B) ** **  **  **   ns
  14        High VN↑(D) – Low VN↓(C)  * ns  ns  ns   ns
  15              High VN↑(D) – Moon **  *  ns  **   ns
  16            Model fluvial – Moon ** **  ns  ns   **
